# Week 2: Hierarchical Data Structure: General Tree - BOJ 1920

In [2]:
class Node:
    def __init__(self, e = None, par = None):
        self.elem = e
        self.height = 0 if e == None else 1
        self.parent = par
        self.left = None if e == None else Node(None, self)
        self.right = None if e == None else Node(None, self)
        
    def isExternal(self):
        if self.left == None and self.right == None:
            return True
        else: return False
    
    def resetHeight(self):
        if self.left.height >= self.right.height:
            self.height = self.left.height + 1
        else: self.height = self.right.height + 1

class AVL_Tree:
    def __init__(self):
        self.root = None
    
    # if direction is True, then direction is right.
    def rotate(self, node : Node, direction : bool = False):
        c = None
        if not direction:
            c = node.right
            node.right = c.left
            c.left.parent = node
            
            c.left = node
            c.parent = node.parent
            if node.parent != None:
                if node.parent.left == node:
                    node.parent.left = c
                else: node.parent.right = c
            node.parent = c
            
        else:
            c = node.left
            node.left = c.right
            c.right.parent = node
            
            c.right = node
            c.parent = node.parent
            if node.parent != None:
                if node.parent.left == node:
                    node.parent.left = c
                else: node.parent.right = c
            node.parent = c

        # reset Heights
        node.resetHeight()
        c.resetHeight()
        # reset Root
        if node == self.root:
            self.root = c
    
    def search(self, e):
        if self.root == None:
            return 0
        
        cur_node = self.root
        while not cur_node.isExternal():
            if e == cur_node.elem:
                return 1
            elif e < cur_node.elem:
                cur_node = cur_node.left
            else:
                cur_node = cur_node.right
        
        return 0
    
    def restructure(self, node : Node):
        z = node
        y = None
        y_dir = None
        x = None
        x_dir = None
        if z.left.height >= z.right.height:
            y, y_dir = z.left, True
        else: y, y_dir = z.right, False
        if y.left.height >= y.right.height:
            x, x_dir = y.left, True
        else: x, x_dir = y.right, False
        # Optimization
        if y.left.height == y.right.height:
            if y_dir:
                x, x_dir = y.left, True
            else: x, x_dir =  y.right, False
        
        # Single Rotation
        if y_dir == x_dir:
            self.rotate(z, y_dir)
        else:
            self.rotate(y, x_dir)
            self.rotate(z, y_dir)
        
        return z
        
    
    def insert(self, e):
        new_node = Node(e)
        
        if self.root == None:
            self.root = new_node
            return
        
        # search correct position
        cur_node = self.root
        while not cur_node.isExternal():
            if e <= cur_node.elem:
                cur_node = cur_node.left
            else:
                cur_node = cur_node.right
        
        # insert the new node
        new_node.parent = cur_node.parent
        if cur_node.parent.left == cur_node:
            cur_node.parent.left = new_node
        else:
            cur_node.parent.right = new_node
        del cur_node
        
        # restructuring
        while new_node != None:
            if abs(new_node.left.height - new_node.right.height) > 1:
                self.restructure(new_node)
            new_node.resetHeight()
            new_node = new_node.parent
    
    def getPredecessor(self, node : Node):
        node = node.left
        while not node.right.isExternal():
            node = node.right
        return node
    
    def remove(self, e):
        if self.root == None:
            print("ERROR: Empty")
            return
        
        # search the node of e
        tmp, rest_node = None, None
        target_node = self.search(e)
        if target_node.isExternal():
            print("ERROR: Not Exist")
            return
        else:
            if target_node.left.isExternal() and target_node.right.isExternal():
                tmp = Node()
            elif target_node.left.isExternal():
                tmp = target_node.right
            elif target_node.right.isExternal():
                tmp = target_node.left
            
            # case 2
            if not target_node.left.isExternal() and not target_node.right.isExternal():
                pred = self.getPredecessor(target_node)
                target_node.elem = pred.elem
                if pred.parent == target_node:
                    target_node.left = pred.left
                else:
                    pred.parent.right = pred.left
                
                rest_node = pred.parent
                del pred
                
            # case 1
            else:
                if target_node.parent.left == target_node: target_node.parent.left = tmp
                else: target_node.parent.right = tmp
                if target_node == self.root: self.root = None
                
                rest_node = target_node.parent
                del target_node
            
        # restructuring
        while rest_node != None:
            if abs(rest_node.left.height - rest_node.right.height) > 1:
                self.restructure(rest_node)
            rest_node.resetHeight()
            rest_node = rest_node.parent

In [3]:
N = int(input())
numbers = list(map(int, input().split()))

avl = AVL_Tree()

for e in numbers:
    avl.insert(e)

M = int(input())
queries = list(map(int, input().split()))

for q in queries:
    print(avl.search(q))

 5
 4 1 5 2 3
 5
 1 3 7 9 5


1
1
0
0
1
